In [ ]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 88.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 117.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=0fa426b166da21e7d8bd608310e560c2f48915ec916ec58777138524c83a1773
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
import os
import json
import pandas as pd
import numpy as np
import time
import torch
from sentence_transformers import SentenceTransformer, util



In [ ]:
with open('evidence.json', 'r') as file:
    evidence_data = json.load(file)
with open('train-claims.json') as file:
    train_data = json.load(file)
with open('test-claims-unlabelled.json') as file:
    test_data = json.load(file)
with open('dev-claims.json') as file:
    dev_data = json.load(file)

In [ ]:
model = SentenceTransformer('bert-base-nli-mean-tokens')
sentences = ["Hello, how are you?", "I am fine. Thanks for asking."]
embeddings = model.encode(sentences)
similarity = util.pytorch_cos_sim(embeddings[0], embeddings[1])

print(f"Similarity: {similarity}")

Similarity: tensor([[0.6091]])


In [ ]:
# sentences = [dev_data[item]['claim_text'] for item in dev_data.keys()]
sentences = [train_data[item]['claim_text'] for item in train_data.keys()]
# sentences = [test_data[item]['claim_text'] for item in test_data.keys()]
# sentences = [evidence_data[item] for item in evidence_data.keys()]

In [ ]:
# Set the device for GPU acceleration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
model = model.to(device)

# Define the sentences
batch_size = 2048   # 1024-80s, 2048-77.4s, 4096-78.5s
num_batches = len(sentences) // batch_size + 1  # 如果加一，batch_size不能整除

start = time.time()
embeddings = np.empty((len(sentences), model.get_sentence_embedding_dimension()))
for i in range(num_batches):
    # Get the current batch
    batch_start = i * batch_size
    batch_end = min((i + 1) * batch_size, len(sentences))
    batch_sentences = sentences[batch_start:batch_end]
    # Encode the batch sentences
    batch_embeddings = model.encode(batch_sentences, convert_to_tensor=True).to(device)
    embeddings[batch_start:batch_end] = batch_embeddings.cpu().numpy()

print(f'{len(sentences)} sentences: ', time.time()-start, ' s')
np.save('train_embeddings.npy', embeddings)


1228 sentences:  2.275181293487549  s


In [ ]:
# load embeddings
# evidence_embedding = np.load('evidence_embeddings.npy', allow_pickle=True)
dev_claim_embedding = np.load('dev_embeddings.npy', allow_pickle=True)
train_claim_embedding = np.load('train_embeddings.npy', allow_pickle=True)
test_claim_embedding = np.load('test_embeddings.npy', allow_pickle=True)
